# SymPy Basics

1. **Symbols**
2. **Equals signs**
3. **Substitution**
4. **evalf**
5. **lambdify**
6. **simplify**
7. **expand**
8. **collect**

## 1. Symbols

Load the SymPy library

In [ ]:
from sympy import *

Now, suppose we start to do a computation.

In [ ]:
x + 1

What happened here? We tried to use the variable x, but it tells us that x is not defined. In Python, variables have no meaning until they are defined. SymPy is no different. Unlike many symbolic manipulation systems you may have used, in SymPy, variables are not defined automatically. To define variables, we must use `symbols`.

In [ ]:
x = symbols('x')
x + 1

`symbols` takes a string of variable names separated by spaces or commas, and creates Symbols out of them. We can then assign these to variable names.

In [ ]:
x, y, z = symbols('x y z')

Finally, let us be sure we understand the difference between SymPy Symbols and Python
variables. Consider the following:

In [ ]:
x = symbols('x')
expr = x + 1
x = 2

What do you think the output of this code will be? If you thought 3, you’re wrong. Let’s see
what really happens

In [ ]:
print(expr)

Changing x to 2 had no effect on expr. This is because `x = 2` changes the Python variable x to 2, but has no effect on the SymPy Symbol x, which was what we used in creating expr.

When we created expr, the Python variable x was a Symbol. After we created, it, we changed the Python variable x to 2. But expr remains the same. All Python programs work this way.

In [ ]:
expr = x + 1
print(expr)

In [ ]:
x = 'abc'

expr = x + 'def'
print('1) ', expr)

x = 'ABC'
print('2) ', expr)

expr = x + 'def'
print('3) ', expr)

## 2. Equals signs

In SymPy, the equals sign `=` works exactly as it does in Python: it is used for variable assignment, not for expressing mathematical equality.

You may think, however, that `==`, which is used for equality testing in Python, is used for SymPy as equality. 

This is not quite correct either. Let us see what happens when we use `==`.

In [ ]:
x = symbols('x')
x + 1 == 4

Instead of treating `x + 1 == 4` symbolically, we just got False. In SymPy, `==` represents exact symbolic equality testing. This means that `a == b` means that we are asking if \(a = b\). We always get a bool as the result of `==`.

There is a separate object, called Eq, which can be used to create symbolic equalities

In [ ]:
Eq(x + 1, 4)

There is one additional caveat about `==` as well. Suppose we want to know if $(x + 1)^2 = x^2 + 2x + 1$. 

In [ ]:
(x + 1)**2 == x**2 + 2*x + 1

We got False again. However, $((x + 1)^2)$ does equal $(x^2 + 2x + 1)$.

Recall from above that `==` represents exact symbolic equality testing. “Exact” here means that two symbolic expressions will compare equal with `==` only if they are exactly equal symbolically. 

Here, $((x + 1)^2)$ and $(x^2 + 2x + 1)$ are not the same symbolically. One is the power of an addition of two terms, and the other is the addition of three terms.

To test if two things are equal, it is best to recall the basic fact that if \(a = b\), then \(a - b = 0\). Thus, the best way to check if \(a = b\) is to take \(a - b\) and simplify it, and see if it goes to 0. 

We will learn later that the function used for this purpose is called simplify. While simplify is very effective for many common expressions, it is important to note that, in general, it is theoretically impossible to determine whether two symbolic expressions are identically equal in all cases. However, for most practical purposes, simplify works quite well.

In [ ]:
a = (x + 1)**2
b = x**2 + 2*x + 1
simplify(a - b)

In [ ]:
c = x**2 - 2*x + 1
simplify(a - c)

## 3. Substitution

Substitution replaces all instances of something in an expression with something else. It is done using the `subs` method.

In [ ]:
from sympy import *
x, y, z = symbols("x y z")

expr = cos(x) + 1
expr.subs(x, y)

Substitution is usually done for one of two reasons:

1) Evaluating an expression at a point. For example, if our expression is $cos(x) + 1$ and we want to evaluate it at the point $x = 0$, so that we get $cos(0) + 1$, which is 2.

In [ ]:
expr.subs(x, 0)

2) Replacing a subexpression with another subexpression. There are two reasons we might want to do this. The first is if we are trying to build an expression that has some symmetry, such as $(x^{x^{x^x}})$. 
To build this, we might start with $x**y$, and replace $y$ with $x**y$. We would then get $x**(x**y)$. If we replaced $y$ in this new expression with $x**x$, we would get $x**(x**(x**x))$, the desired expression.

In [ ]:
expr = x**y
print(expr)

expr = expr.subs(y, x**y)
print(expr)

expr = expr.subs(y, x**x)
print(expr)

There are two important things to note about subs. First, it returns a new expression. **SymPy objects are immutable**. That means that subs does modify it in-place.

In [ ]:
expr = cos(x)
expr.subs(x, 0)

In [ ]:
expr

In [ ]:
x

To perform multiple substitutions at once, pass a list of (old, new) pairs to `subs`.

In [ ]:
expr = x**3 + 4*x*y - z
expr.subs([(x, 2), (y, 4), (z, 0)])

## 4. evalf

To evaluate a numerical expression into a floating point number, use `evalf`.

In [ ]:
expr = sqrt(8)
expr.evalf()

SymPy can evaluate floating point expressions to arbitrary precision. By default, 15 digits of precision are used, but you can pass any number as the argument to `evalf`. Let’s compute the first 100 digits of $(\pi)$.

In [ ]:
pi.evalf()

In [ ]:
pi.evalf(100)

To numerically evaluate an expression with a Symbol at a point, we might use subs followed by `evalf`:

In [ ]:
expr = cos(2*x)

expr.subs(x, 2.4).evalf()

But it is more efficient and numerically stable to pass the substitution to evalf using the subs flag, which takes a dictionary of Symbol: point pairs.

In [ ]:
expr.evalf(subs = {x: 2.4})

## 5. lambdify

`subs` and `evalf` are good if you want to do simple evaluation, but if you intend to evaluate an expression at many points, there are more efficient ways. 

For example, if you wanted to evaluate an expression at a thousand points, using SymPy would be far slower than it needs to be, especially if you only care about machine precision. 
Instead, you should use libraries like NumPy and SciPy.

The easiest way to convert a SymPy expression to an expression that can be numerically evaluated is to use the `lambdify` function. 
`lambdify` acts like a lambda function, except it converts the SymPy names to the names of the given numerical library, usually NumPy.

In [ ]:
import numpy

a = numpy.linspace(0, 2 * numpy.pi, 20)

print("Evaluating expression at points in array:")
print(a)

x = symbols('x')
expr = sin(x)

f = lambdify(x, expr, "numpy")
results_numpy = f(a)

print("Results using NumPy:")
print(results_numpy)

You can use other libraries than **NumPy**. For example, to use the standard library **math** module, use "math".

In [ ]:
f = lambdify(x, expr, "math")
f(a)

When using the standard library math with `lambdify`, the generated function only accepts scalar values, not arrays.

In [ ]:
f(0.1)

To evaluate the same expression using the standard library math module with `lambdify`, you need to use a for loop, since the generated function only accepts scalar values, not arrays.

This approach evaluates the function at each point in the array individually.

In [ ]:
results_math = []
for val in a:
    results_math.append(f(val))

print("Results using math:")
print(results_math)

We can compare the two sets of results, `results_numpy` and `results_math`, to verify that both approaches produce the same values for evaluating the expression at multiple points. This helps ensure consistency between vectorized and scalar evaluations.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 5))
plt.plot(a, results_math, label='results_math', marker='o')
plt.plot(a, results_numpy, label='results_numpy', marker='x')
plt.legend()
plt.title('Comparison of between math and NumPy evaluations')
plt.xlabel('x')
plt.ylabel('sin(x)')
plt.show()

## 6. simplify

One of the most useful features of a symbolic manipulation system is the ability to simplify mathematical expressions. SymPy provides a function called `simplify` that can be used to simplify expressions.

In [ ]:
x = symbols('x')

# example 1
expr1 = sin(x)**2 + cos(x)**2
print("Simplify function:")
print(f"simplify({expr1}) = {simplify(expr1)}")

# example 2
expr2 = (x**3 + x**2 - x - 1)/(x**2 + 2*x + 1)
print(f"simplify({expr2}) = {simplify(expr2)}")


## 7. expand

Given a polynomial, `expand` will put it into a canonical form of a sum of monomials.

In [ ]:
expand((x + 1)**2)

In [ ]:
expand((x + 2)*(x - 3))

In [ ]:
expand((x + 1)*(x - 2) - (x - 1)*x)

## 8. collect

`collect` collects common powers of a term in an expression.

In [ ]:
expr = x*y + x - 3 + 2*x**2 - z*x**2 + x**3
expr

In [ ]:
collected_expr = collect(expr, x)
collected_expr

`collect` is particularly useful in conjunction with the `.coeff` method. 

`expr.coeff(x, n)` gives the coefficient of `x**n` in `expr`:

In [ ]:
collected_expr.coeff(x, 2)